In [ ]:
from typing import List, Type
from pydantic import BaseModel
import json
import requests

# ---------------------------
# 1. Define the Pydantic Schema
# ---------------------------
class ParsedRequest(BaseModel):
    intent: str                 # "table", "qualitative", or "in-depth"
    banks: List[str]            # Must match allowed_banks
    quarters: List[str]         # e.g., "1Q2025", "4Q2024"
    metrics: List[str]          # Must match allowed_metrics


# ---------------------------
# 2. Ollama-based LLM Parser
# ---------------------------
def parse_with_ollama(
    llm_model_name: str,
    user_input: str,
    schema: Type[BaseModel],
    allowed_banks: List[str],
    allowed_metrics: List[str],
    ollama_host: str = "http://localhost:11434/api/chat"
) -> BaseModel:
    system_prompt = f"""
You are a financial assistant that converts natural language input into structured JSON.

Output format:
{{
  "intent": "table" | "qualitative" | "in-depth",
  "banks": [valid bank names],
  "quarters": ["1Q2025", "4Q2024", "3Q2024"],
  "metrics": [valid metrics]
}}

Rules:
- intent must be one of: "table", "qualitative", or "in-depth"
- banks must match only from: {json.dumps(allowed_banks)}
- metrics must match only from: {json.dumps(allowed_metrics)}
- Convert "Q1 2025" or "1st quarter of 2025" to "1Q2025"
- Always include the requested quarter and the previous two quarters
"""

    # Prepare Ollama message format
    payload = {
        "model": llm_model_name,
        "messages": [
            {"role": "system", "content": system_prompt.strip()},
            {"role": "user", "content": user_input.strip()}
        ],
        "stream": False
    }

    # Call Ollama
    response = requests.post(ollama_host, json=payload)
    response.raise_for_status()
    content = response.json()["message"]["content"]

    try:
        return schema.parse_raw(content)
    except Exception as e:
        raise ValueError(f"Failed to parse LLM output: {e}\nRaw Output:\n{content}")


In [ ]:
if __name__ == "__main__":
    # Define your bank/metric lists
    allowed_banks = ["JP Morgan Chase", "Bank of America", "Citigroup", "Wells Fargo"]
    allowed_metrics = ["EarningsPerShare", "NetIncome", "TotalRevenue", "ReturnOnEquity"]

    # Your input
    user_input = "Show me EPS and net income of JPMC and Citi for Q1 2025 in a table"

    # Parse using Ollama
    result = parse_with_ollama(
        llm_model_name="llama3",  # or "mistral" or any model you loaded
        user_input=user_input,
        schema=ParsedRequest,
        allowed_banks=allowed_banks,
        allowed_metrics=allowed_metrics
    )

    # Show result
    print(result.json(indent=2))
